# **1. Install Elasticsearch in Python**

In [32]:
pip install --upgrade elasticsearch


# **2. Connect to Elasticsearch(test connection)**

In [33]:
from elasticsearch import Elasticsearch

import os
from elasticsearch import Elasticsearch

ELASTIC_ENDPOINT = os.getenv("ELASTIC_ENDPOINT")
ELASTIC_API_KEY = os.getenv("ELASTIC_API_KEY")

client = Elasticsearch(
    "https://100084975667404d84215c661a178882.us-central1.gcp.cloud.es.io:443",
    api_key="c01INlJad0JhQVJZcmtiMmdDMEk6d3FrQ3diM08yZE5lLU50cWhqWklRdw=="
)

# Test connection
print(client.info())

{'name': 'instance-0000000000', 'cluster_name': '100084975667404d84215c661a178882', 'cluster_uuid': '46RCnWz5Q3mD7kkQnqbpYg', 'version': {'number': '9.3.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '17b451d8979a29e31935fe1eb901310350b30e62', 'build_date': '2026-01-29T10:05:46.708397977Z', 'build_snapshot': False, 'lucene_version': '10.3.2', 'minimum_wire_compatibility_version': '8.19.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'}


In [34]:
import os

os.environ["ELASTIC_ENDPOINT"] = "https://100084975667404d84215c661a178882.us-central1.gcp.cloud.es.io:443"
os.environ["ELASTIC_API_KEY"] = "c01INlJad0JhQVJZcmtiMmdDMEk6d3FrQ3diM08yZE5lLU50cWhqWklRdw=="


# **3. Index creation( A place to store data)**

In [35]:
index_name = "national-parkss-v2"

if not client.indices.exists(index=index_name):
    client.indices.create(
        index=index_name,
        mappings={
            "properties": {
                "text": {
                    "type": "text"
                }
            }
        }
    )
    print("New index created")
else:
    print(f"Index '{index_name}' already exists")

Index 'national-parkss-v2' already exists


# **4. Ingest document into the index**

In [36]:
docs = [
    {
        "_index": "national-parks-v2",
        "text": "Yellowstone National Park is famous for geysers and wildlife like bison and wolves."
    },
    {
        "_index": "national-parks-v2",
        "text": "Yosemite National Park is known for granite cliffs, waterfalls, and giant sequoia trees in the Sierra Nevada."
    },
    {
        "_index": "national-parks-v2",
        "text": "Rocky Mountain National Park features alpine tundra, mountain peaks, and diverse wildlife."
    }
]

from elasticsearch import helpers

helpers.bulk(
    client,
    docs,
    refresh="wait_for"
)

print("Documents indexed")


Documents indexed


# 5. **Classic Elasticsearch**

In [37]:
response = client.search(
    index="national-parks-v2",
    query={
        "match": {
            "text": "Sierra Nevada"
        }
    }
)

for hit in response["hits"]["hits"]:
    print(hit["_source"]["text"])


Yosemite National Park is known for granite cliffs, waterfalls, and giant sequoia trees in the Sierra Nevada.
Yosemite National Park is known for granite cliffs, waterfalls, and giant sequoia trees in the Sierra Nevada.


# **Recreation**

## **Ingestion of documents**

In [38]:
from elasticsearch import helpers

docs = [
    {
        "_index": "national-parks-v2",
        "text": "Yellowstone National Park is famous for geysers and wildlife like bison and wolves."
    },
    {
        "_index": "national-parks-v2",
        "text": "Yosemite National Park is known for granite cliffs, waterfalls, and giant sequoia trees in the Sierra Nevada."
    },
    {
        "_index": "national-parks-v2",
        "text": "Rocky Mountain National Park features alpine tundra, mountain peaks, and diverse wildlife."
    }
]

helpers.bulk(
    client,
    docs,
    refresh="wait_for"
)

print("Documents indexed successfully")


Documents indexed successfully


## **Search the data**

In [39]:
response = client.search(
    index="national-parks-v2",
    query={
        "match": {
            "text": "Sierra Nevada"
        }
    }
)

for hit in response["hits"]["hits"]:
    print(hit["_source"]["text"])


Yosemite National Park is known for granite cliffs, waterfalls, and giant sequoia trees in the Sierra Nevada.
Yosemite National Park is known for granite cliffs, waterfalls, and giant sequoia trees in the Sierra Nevada.
Yosemite National Park is known for granite cliffs, waterfalls, and giant sequoia trees in the Sierra Nevada.


In this, when running the indexing code each time, it added the same 3 documents again. Elasticsearch does not automatically prevent duplicates unless, providing our own document IDs or delete the index before re-indexing. Right now in the current index, it probably contains:

* Yosemite(multiple copies)
* Yellowstone(multiple copies)
* Rocky Mountain(multiple copies)

To fix this properly,there have to delete the index, recreate it and reinsert the documents with fixed IDs. This prevents the duplication.






## **1. Delete the index**

In [40]:
client.indices.delete(index="national-parks-v2")
print("Index deleted successfully")


Index deleted successfully


## **2. Recreate the index**

In [41]:
client.indices.create(
    index="national-parks-v2",
    mappings={
        "properties": {
            "text": {
                "type": "text"
            }
        }
    }
)

print("Index recreated successfully")


Index recreated successfully


## **3. Insertion of documents with IDs**

In [42]:
from elasticsearch import helpers

docs = [
    {
        "_index": "national-parks-v2",
        "_id": "1",
        "text": "Yellowstone National Park is famous for geysers and wildlife like bison and wolves."
    },
    {
        "_index": "national-parks-v2",
        "_id": "2",
        "text": "Yosemite National Park is known for granite cliffs, waterfalls, and giant sequoia trees in the Sierra Nevada."
    },
    {
        "_index": "national-parks-v2",
        "_id": "3",
        "text": "Rocky Mountain National Park features alpine tundra, mountain peaks, and diverse wildlife."
    }
]

helpers.bulk(client, docs, refresh="wait_for")

print("Documents indexed cleanly")


Documents indexed cleanly


## **4. Performing Search again**

In [43]:
response = client.search(
    index="national-parks-v2",
    query={
        "match": {
            "text": "Sierra Nevada"
        }
    }
)

for hit in response["hits"]["hits"]:
    print(hit["_source"]["text"])


Yosemite National Park is known for granite cliffs, waterfalls, and giant sequoia trees in the Sierra Nevada.
